In [1]:
from nerfstudio.pipelines.imaginedriving_pipeline import ImagineDrivingPipelineConfig, ImagineDrivingPipeline
from nerfstudio.data.datamanagers.ad_datamanager import ADDataManagerConfig, ADDataManager
from nerfstudio.data.dataparsers.pandaset_dataparser import PandaSetDataParserConfig

from nerfstudio.cameras.camera_optimizers import CameraOptimizerConfig
from nerfstudio.models.neurad import NeuRADModelConfig

In [2]:
%load_ext autoreload
%autoreload 2
%cd /home/s0001899/Dev/neurad-studio

/home/s0001899/Dev/neurad-studio


In [3]:
from dataclasses import asdict

config = ImagineDrivingPipelineConfig(
    datamanager=ADDataManagerConfig(
        dataparser=PandaSetDataParserConfig(add_missing_points=True)
    ),
    model=NeuRADModelConfig(
        eval_num_rays_per_chunk=1 << 15,
        camera_optimizer=CameraOptimizerConfig(mode="off"),  # SO3xR3
    ),
)
asdict(config)

{'_target': nerfstudio.pipelines.imaginedriving_pipeline.ImagineDrivingPipeline,
 'datamanager': {'_target': nerfstudio.data.datamanagers.ad_datamanager.ADDataManager,
  'data': None,
  'masks_on_gpu': False,
  'images_on_gpu': False,
  'dataparser': {'_target': nerfstudio.data.dataparsers.pandaset_dataparser.PandaSet,
   'data': PosixPath('data/pandaset'),
   'sequence': '001',
   'train_split_fraction': 0.5,
   'train_eval_split_type': <SplitTypes.LINSPACE: 'linspace'>,
   'max_eval_frames': None,
   'dataset_start_fraction': 0.0,
   'dataset_end_fraction': 1.0,
   'cameras': ('front', 'front_left', 'front_right', 'back', 'left', 'right'),
   'lidars': ('Pandar64',),
   'min_lidar_dist': (1.0, 2.0, 2.0),
   'radars': (),
   'load_cuboids': True,
   'include_deformable_actors': False,
   'annotation_interval': 0.1,
   'rolling_shutter_offsets': (-0.03, 0.01),
   'allow_per_point_times': True,
   'add_missing_points': True,
   'lidar_elevation_mapping': {'Pandar64': {0: 14.882,
     1:

In [5]:
pipe = ImagineDrivingPipeline(config, device='cpu')

Variable resolution, using variable_res_collate

Output()

Output()

Started processes


Setting up evaluation dataset...

...

4D hashgrid is not supported with torch implementation, falling back multiple grids.

4D hashgrid is not supported with torch implementation, falling back multiple grids.

4D hashgrid is not supported with torch implementation, falling back multiple grids.

/home/s0001899/Dev/neurad-studio/neurad-env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/s0001899/Dev/neurad-studio/neurad-env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/s0001899/Dev/neurad-studio/neurad-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
pipe.datamanager

ADDataManager(
  (train_lidar_ray_generator): LidarRayGenerator()
  (eval_ray_generator): RayGenerator()
  (eval_lidar_ray_generator): LidarRayGenerator()
)